In [1]:
from cryptofeed.defines import TRADES, L2_BOOK, BID, ASK

import pandas as pd
import json

import time
import datetime

import threading
import schedule
import redis

from algos.altcoin.backtest import perform_backtests
from algos.altcoin.live_trader import liveTrading, round_down
from utils import print


In [17]:
r = redis.Redis(host='localhost', port=6379, db=0)
config = pd.read_csv('algos/altcoin/config.csv')
details_df = pd.DataFrame()

for idx, row in config.iterrows():
    asset = row['name']

    curr_detail = pd.Series()
    curr_detail['name'] = asset
    backtest = pd.read_csv("data/trades_{}.csv".format(asset))

    try:
        curr_detail['position'] = r.get('{}_current_pos'.format(asset)).decode()
    except:
        curr_detail['position'] = "NONE"

    try:
        curr_detail['entry'] = round(float(r.get('{}_avgEntryPrice'.format(asset)).decode()),2)
    except:
        curr_detail['entry'] = 0

    try:
        curr_detail['pos_size'] = float(r.get('{}_pos_size'.format(asset)).decode())
    except:
        curr_detail['pos_size'] = 0

    try:
        curr_detail['live_price'] = round(float(r.get('{}_best_ask'.format(asset)).decode()), 3)
    except:
        curr_detail['live_price'] = 0


    curr_detail['backtest_position'] = 'SHORT' if backtest.iloc[-1]['Type'] == 'SELL' else 'LONG'
    curr_detail['backtest_date'] = backtest.iloc[-1]['Date']
    curr_detail['entry_price'] = round(backtest.iloc[-1]['Price'], 2)
    curr_detail['to_trade'] = row['to_trade']
    curr_detail['live_lev'] = int(row['mult'])

    try:
        curr_detail['live_pnl'] = round(((curr_detail['live_price'] - curr_detail['entry'])/curr_detail['entry']) * 100 * curr_detail['live_lev'], 2)

        if curr_detail['backtest_position'] == 'SHORT':
            curr_detail['live_pnl'] = curr_detail['live_pnl'] * -1
    except:
        curr_detail['live_pnl'] = 0
        
    try:
        curr_detail['backtest_pnl'] = round(((curr_detail['live_price'] - curr_detail['entry_price'])/curr_detail['entry_price']) * 100 * curr_detail['live_lev'], 2)

        if curr_detail['backtest_position'] == 'SHORT':
            curr_detail['backtest_pnl'] = curr_detail['backtest_pnl'] * -1
    except:
        curr_detail['live_pnl'] = 0
        
    curr_detail['allocation'] = row['allocation']

    details_df = details_df.append(curr_detail, ignore_index=True)
    #need backtest pnl too

<ipython-input-17-4a283b97c145>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  curr_detail = pd.Series()


In [22]:
details_df

,allocation,backtest_date,backtest_pnl,backtest_position,entry,entry_price,live_lev,live_pnl,live_price,name,pos_size,position,to_trade
0,0.200,2020-12-11,-55.71,SHORT,0.0,0.14,2.0,0.0,0.179,ADA-PERP,0.0,NONE,1.0
1,0.034,2020-11-28,-9.37,SHORT,0.0,13.19,3.0,0.0,13.602,BAL-PERP,0.0,NONE,1.0
2,0.005,2020-12-25,-1.15,SHORT,0.0,161.98,3.0,0.0,162.600,BSV-PERP,0.0,NONE,1.0
3,0.011,2020-12-18,47.60,LONG,0.0,22836.78,3.0,0.0,26460.000,BTC-PERP,0.0,NONE,1.0
4,0.000,2020-12-20,NaN,LONG,0.0,0.00,2.0,0.0,0.004,DOGE-PERP,0.0,NONE,1.0
5,0.100,2020-12-11,-3.64,SHORT,0.0,1097.09,3.0,0.0,1110.400,DRGN-PERP,0.0,NONE,1.0
6,0.000,2020-12-05,18.17,SHORT,0.0,2.84,3.0,0.0,2.668,EOS-PERP,0.0,NONE,1.0
7,0.040,2020-10-28,-12.63,SHORT,0.0,5.51,3.0,0.0,5.742,ETC-PERP,0.0,NONE,1.0
8,0.070,2020-12-29,-10.77,LONG,0.0,730.36,3.0,0.0,704.130,ETH-PERP,0.0,NONE,1.0
9,0.080,2020-12-09,-0.15,SHORT,0.0,12.14,2.0,0.0,12.149,LINK-PERP,0.0,NONE,1.0


In [23]:
(details_df['backtest_pnl'] * details_df['allocation']).sum()

13.454069999999998

In [13]:
details_df['entry_price']

0         0.14
1        13.19
2       161.98
3     22836.78
4         0.00
5      1097.09
6         2.84
7         5.51
8       730.36
9        12.14
10      997.70
11     1453.25
12        2.59
13        0.85
14        0.52
Name: entry_price, dtype: float64

In [15]:
details_df['live_price']

0         0.179
1        13.602
2       162.600
3     26460.000
4         0.004
5      1110.400
6         2.668
7         5.742
8       704.130
9        12.149
10      974.500
11     1398.900
12        2.491
13        0.677
14        0.224
Name: live_price, dtype: float64

In [3]:
details_df

,backtest_date,backtest_position,entry,entry_price,live_lev,live_pnl,live_price,name,pos_size,position,to_trade
0,2020-12-11,SHORT,0.0,0.14,2.0,0.0,0.179,ADA-PERP,0.0,NONE,1.0
1,2020-11-28,SHORT,0.0,13.19,3.0,0.0,13.602,BAL-PERP,0.0,NONE,1.0
2,2020-12-25,SHORT,0.0,161.98,3.0,0.0,162.600,BSV-PERP,0.0,NONE,1.0
3,2020-12-18,LONG,0.0,22836.78,3.0,0.0,26460.000,BTC-PERP,0.0,NONE,1.0
4,2020-12-20,LONG,0.0,0.00,2.0,0.0,0.004,DOGE-PERP,0.0,NONE,1.0
5,2020-12-11,SHORT,0.0,1097.09,3.0,0.0,1110.400,DRGN-PERP,0.0,NONE,1.0
6,2020-12-05,SHORT,0.0,2.84,3.0,0.0,2.668,EOS-PERP,0.0,NONE,1.0
7,2020-10-28,SHORT,0.0,5.51,3.0,0.0,5.742,ETC-PERP,0.0,NONE,1.0
8,2020-12-29,LONG,0.0,730.36,3.0,0.0,704.130,ETH-PERP,0.0,NONE,1.0
9,2020-12-09,SHORT,0.0,12.14,2.0,0.0,12.149,LINK-PERP,0.0,NONE,1.0
